In [1]:
import os
import regex as re
import csv
import shutil

In [2]:
# Variables
source_dir = "source_files/"
source_metadata_file = source_dir + "papers.csv"
processed_metadata_file = source_dir + "papers_processed.csv"

table_output_dir = "extracted_tables/"
table_code_dir = table_output_dir + "table_code/"
table_result_file = table_output_dir + "tables.csv"

figure_output_dir = "extracted_figures/"
figure_result_file = figure_output_dir + "figures.csv"

possible_extensions = [".pdf", ".png", ".jpg", ".jpeg", ".eps", ".ps"]
WORD_THRESHOLD = 40
MAX_PROCESSING_TIME = 30

os.makedirs(table_output_dir, exist_ok=True)
os.makedirs(table_code_dir, exist_ok=True)
os.makedirs(figure_output_dir, exist_ok=True)

if os.path.isfile(table_result_file) == False:
    open(table_result_file, "w").close()
    
if os.path.isfile(figure_result_file) == False:
    open(figure_result_file, "w").close()
    
if os.path.isfile(processed_metadata_file) == False:
    open(processed_metadata_file, "w").close()

In [3]:
"""
Returns a list of papers that were downloaded but not yet processed.
"""
def get_unprocessed_papers():
    unprocessed_papers = []
    with open(source_metadata_file, "r") as csvfile_source:
        with open(processed_metadata_file, "r") as csvfile_processed:
            source_spamreader = csv.reader(csvfile_source, delimiter=';', quotechar='|', quoting=csv.QUOTE_ALL)
            processed_spamreader = csv.reader(csvfile_processed, delimiter=';', quotechar='|', quoting=csv.QUOTE_ALL)

            processed_papers = {row[0] for row in processed_spamreader}

            for row in source_spamreader:
                if row[0] not in processed_papers:
                    unprocessed_papers.append(row)
    
    return unprocessed_papers

In [4]:
"""
Extracting tables and figures from the downloaded papers in the source directory
"""
def extract_data_from_papers(unprocessed_papers, amount):
    for i in range(amount):
        row = unprocessed_papers[i]
        
        paper_id = row[0]
        paper_path = source_dir + paper_id
        if os.path.isdir(paper_path):
            print("\n\n", i, paper_id)
            tex_files = [os.path.join(root, file)
                 for root, dirs, files in os.walk(paper_path)
                 for file in files if file.endswith(".tex") and file.startswith("FR_") == False]
            print(tex_files)

            complete_tex = ""
            for file in tex_files:
                try:
                    f = open(file, "r", encoding="utf8")
                    complete_tex += f.read()
                    f.close()
                except UnicodeDecodeError as e:
                    print("UnicodeDecodeError occurred. File could not be loaded.")
                    continue
            
            try:
            
                # Search for table and figure structures
                found_tables = re.findall(r"\\begin\{table\*?\}.*?\\end\{table\*?\}", complete_tex, re.DOTALL, timeout=MAX_PROCESSING_TIME)
                found_figures = re.findall(r"\\begin\{figure\*?\}.*?\\end\{figure\*?\}", complete_tex, re.DOTALL, timeout=MAX_PROCESSING_TIME)
                counter_table = 0
                counter_figure = 0

                # Capture document preamble. If it can't be captured, skip table capturing.
                found_document_header = re.search(r"\\documentclass\[.*?\\begin\{document\}", complete_tex, re.DOTALL, timeout=MAX_PROCESSING_TIME)
                if found_document_header is None:
                    print("Document header could not be identified.")
                    found_tables = []

                # Table capturing
                for table in found_tables:
                    # Capturing and saving process
                    if capturing_process(table, "TABLE", paper_id, counter_table, complete_tex, document_header=found_document_header.group(0)):
                        counter_table += 1

                # Figure capturing
                for figure in found_figures:
                    # Locating graphic and skipping multi figures
                    found_graphics = re.findall(r"\\includegraphics(\[.*?\])*\{(.*?)\}", figure, timeout=MAX_PROCESSING_TIME)
                    if len(found_graphics) != 1:
                        continue

                    # Capturing and saving process
                    if capturing_process(figure, "FIGURE", paper_id, counter_figure, complete_tex, found_graphic=found_graphics[0]):
                        counter_figure += 1

                number_tables = len(found_tables)
                number_figures = len(found_figures)
                print(f"{counter_table}/{number_tables} tables collected.")
                print(f"{counter_figure}/{number_figures} figures collected.")

                # Storing information in final metadata file and removing the paper from disk
                finish_process(row, counter_table, counter_figure)
            except re.error as e:
                print(f"Regex error for {paper_id}: {e}")
        else:
            print(f"Paper {paper_id} could not be found.")
            finish_process(row, -1, -1)

# Capturing label, paragraph mentions and saving them into metadata file
def capturing_process(current_object, object_type, paper_id, counter, complete_tex, found_graphic=None, document_header=None):
    label_match = re.search(r"\\label\{(.*?)\}", current_object, timeout=MAX_PROCESSING_TIME)
    caption_match = re.search(r"\\caption\{.*\}", current_object, timeout=MAX_PROCESSING_TIME)
    paper_path = source_dir + paper_id

    if label_match and caption_match:
        found_caption = caption_match.group(0)
        found_label = label_match.group(1)
        
        # Skipping if no paragraph mention was found
        found_paragraphs = re.findall(fr".*\\ref\{{{re.escape(found_label)}}}.*", complete_tex, timeout=MAX_PROCESSING_TIME)
        if len(found_paragraphs) == 0:
            return False

        # Combining paragraph mentions
        paragraph_mentions = []
        for paragraph in found_paragraphs:
            paragraph_mentions.append(paragraph)
        paragraph_mentions = " ".join(paragraph_mentions)
            
        # Removing paragraph mentions if word length under threshold
        word_length = len(paragraph_mentions.split())
        if word_length < WORD_THRESHOLD:
            return False
        
        # Set object-related variables
        object_id = None
        result_file = None
        if object_type == "TABLE":
            object_id = paper_id + "_TAB_" + str(counter + 1)
            result_file = table_result_file
        elif object_type == "FIGURE":
            object_id = paper_id + "_FIG_" + str(counter + 1)
            result_file = figure_result_file

        # Object-related saving process
        if object_type == "TABLE":
            # Store as txt file
            table_file_path = table_code_dir + object_id + ".tex"
            table_tex_code = document_header + "\n\pagenumbering{gobble}\n" + current_object + "\n\end{document}"
            f = open(table_file_path, "w", encoding="utf-8")
            f.write(table_tex_code)
            f.close()
        elif object_type == "FIGURE":
            # Locate graphic path
            graphic_path = found_graphic[1]
            file_type = os.path.splitext(graphic_path)[-1]
            
            # Handling the case where the file type is not included in tex code
            if os.path.isfile(paper_path+"/"+graphic_path):
                graphic_path = paper_path+"/"+graphic_path
            else:
                unknown_extension = True
                for ext in possible_extensions:
                    possible_path = paper_path+"/"+graphic_path + ext
                    if os.path.isfile(possible_path):
                        graphic_path = possible_path
                        file_type = ext
                        unknown_extension = False
                        break
                if unknown_extension:
                    return False
            
            # Moving figure file to own dataset
            try:
                os.rename(graphic_path, figure_output_dir + object_id + file_type)
            except FileNotFoundError as e:
                print(f"File not found: {graphic_path} - {e}")
                return False
            except Exception as e:
                print(f"An error occurred: {e}")
                return False
        
        # Store meta information in csv file
        with open(result_file, 'a', newline='', encoding="utf-8") as csvfile:
            spamwriter = csv.writer(csvfile, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            spamwriter.writerow([object_id, paper_id, "https://arxiv.org/abs/" + paper_id, found_caption, paragraph_mentions])
        
        return True
            
# Store meta information in processed_papers csv file and deletes paper from disk if it is not longer needed
def finish_process(row, number_tables, number_figures):    
    # Write into processed_papers csv file
    with open(processed_metadata_file, "a", newline='', encoding="utf-8") as csvfile_processed:
        spamwriter = csv.writer(csvfile_processed, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        new_row = row + [str(number_tables), str(number_figures)]
        spamwriter.writerow(new_row)

    # Delete paper
    try:
        shutil.rmtree(source_dir + row[0])
    except Exception as e:
        print(f"Paper {row[0]} could not be deleted.")
        print(f"Error Type: {type(e).__name__}")
        error_message = str(e)[:100]
        print(f"Error Message: {error_message}")

In [5]:
## Get list of unprocessed papers
list_unprocessed_papers = get_unprocessed_papers()
print(f"Number of unprocessed papers: {len(list_unprocessed_papers)}")

Number of unprocessed papers: 1719


In [6]:
# Set how many processed papers should be processed
number_of_papers_to_process = 1719

# Start the processing process, including extraction
extract_data_from_papers(list_unprocessed_papers, number_of_papers_to_process)



 0 2310.05853
['source_files/2310.05853\\Exploring_User_Perceptions_of_Using_An_LLM-Based_Conversational_Assistant_Toward_Cooking_Partner.tex']
4/8 tables collected.
3/8 figures collected.


 1 2310.05868
['source_files/2310.05868\\acknowledgments.tex', 'source_files/2310.05868\\acronym.tex', 'source_files/2310.05868\\cas-sc-template.tex', 'source_files/2310.05868\\conclusions.tex', 'source_files/2310.05868\\discussion.tex', 'source_files/2310.05868\\experiments_and_results.tex', 'source_files/2310.05868\\introduction.tex', 'source_files/2310.05868\\materials.tex', 'source_files/2310.05868\\model.tex']
0/0 tables collected.
5/5 figures collected.


 2 2310.05871
['source_files/2310.05871\\template.tex']
Document header could not be identified.
0/0 tables collected.
0/5 figures collected.


 3 2310.05881
['source_files/2310.05881\\emnlp.tex']
4/6 tables collected.
5/6 figures collected.


 4 2310.05911
['source_files/2310.05911\\main.tex']
0/0 tables collected.
4/4 figures collected.


Document header could not be identified.
0/0 tables collected.
0/30 figures collected.


 15 2304.01002
['source_files/2304.01002\\abstract.tex', 'source_files/2304.01002\\acknowledgment.tex', 'source_files/2304.01002\\conclusion.tex', 'source_files/2304.01002\\discussion.tex', 'source_files/2304.01002\\ethics.tex', 'source_files/2304.01002\\introduction.tex', 'source_files/2304.01002\\limitation.tex', 'source_files/2304.01002\\literature.tex', 'source_files/2304.01002\\main.tex', 'source_files/2304.01002\\method.tex', 'source_files/2304.01002\\method_copy.tex', 'source_files/2304.01002\\result.tex', 'source_files/2304.01002\\settings.tex', 'source_files/2304.01002\\title.tex']
10/21 tables collected.
3/18 figures collected.


 16 2304.09654
['source_files/2304.09654\\algorithm-exact.tex', 'source_files/2304.09654\\introduction.tex', 'source_files/2304.09654\\preliminaries.tex', 'source_files/2304.09654\\temporal-cm.tex', 'source_files/2304.09654\\tgs-main.tex', 'source_files/2304.0965

0/0 tables collected.
1/15 figures collected.


 32 2306.08749
['source_files/2306.08749\\1_introduction_v1.tex', 'source_files/2306.08749\\2_methods_v1.tex', 'source_files/2306.08749\\3_experimental_design_results_v1.tex', 'source_files/2306.08749\\4_discussion_and_conclusion_v1.tex', 'source_files/2306.08749\\5_supplementary_material_v1.tex', 'source_files/2306.08749\\paper843.tex']
1/3 tables collected.
1/4 figures collected.


 33 2306.10502
['source_files/2306.10502\\neurips_2023.tex']
Document header could not be identified.
0/0 tables collected.
1/7 figures collected.


 34 2306.11375
['source_files/2306.11375\\main.tex', 'source_files/2306.11375\\supporting_info.tex']
2/5 tables collected.
1/4 figures collected.


 35 2306.12369
['source_files/2306.12369\\main.tex']
2/5 tables collected.
2/10 figures collected.


 36 2307.05330
['source_files/2307.05330\\main.tex']
Document header could not be identified.
0/0 tables collected.
0/17 figures collected.


 37 2307.06077
['source_f

0/4 tables collected.
0/7 figures collected.


 53 2310.04373
['source_files/2310.04373\\crlhf_iclr.tex', 'source_files/2310.04373\\imports.tex', 'source_files/2310.04373\\m.macros.tex', 'source_files/2310.04373\\m.symbols.tex']
Document header could not be identified.
0/0 tables collected.
0/12 figures collected.


 54 2310.04490
['source_files/2310.04490\\Diffusion.tex']
0/1 tables collected.
0/2 figures collected.


 55 2310.04567
['source_files/2310.04567\\main.tex']
Document header could not be identified.
0/0 tables collected.
1/4 figures collected.


 56 2310.04570
['source_files/2310.04570\\main.tex', 'source_files/2310.04570\\math_commands.tex', 'source_files/2310.04570\\tex\\abstract.tex', 'source_files/2310.04570\\tex\\approach.tex', 'source_files/2310.04570\\tex\\conclusions.tex', 'source_files/2310.04570\\tex\\experiments.tex', 'source_files/2310.04570\\tex\\introduction.tex', 'source_files/2310.04570\\tex\\relatedworks.tex']
1/2 tables collected.
1/4 figures collected.




0/0 tables collected.
0/2 figures collected.


 86 2310.06122
['source_files/2310.06122\\samplepaper.tex']
0/0 tables collected.
3/3 figures collected.


 87 2310.06123
['source_files/2310.06123\\appendix.tex', 'source_files/2310.06123\\conclusion.tex', 'source_files/2310.06123\\experiment.tex', 'source_files/2310.06123\\introduction.tex', 'source_files/2310.06123\\main.tex', 'source_files/2310.06123\\math_commands.tex', 'source_files/2310.06123\\method.tex', 'source_files/2310.06123\\related_work.tex', 'source_files/2310.06123\\figures\\ablation.tex', 'source_files/2310.06123\\figures\\fl_structure.tex', 'source_files/2310.06123\\figures\\prompt_embs.tex', 'source_files/2310.06123\\figures\\tpg.tex', 'source_files/2310.06123\\tables\\main_table.tex', 'source_files/2310.06123\\tables\\table_ncls.tex', 'source_files/2310.06123\\tables\\table_other_cls.tex', 'source_files/2310.06123\\tables\\table_other_dataset.tex', 'source_files/2310.06123\\tables\\table_transfer.tex']
Document header 

...

7/9 tables collected.
0/12 figures collected.


 1603 2209.05773
['source_files/2209.05773\\CAIBC-mmfp1288.tex', 'source_files/2209.05773\\camera ready mmfp1288 arxiv\\CAIBC-mmfp1288.tex']
8/8 tables collected.
6/12 figures collected.


 1604 2209.05774
['source_files/2209.05774\\main.tex', 'source_files/2209.05774\\supp.tex']
7/7 tables collected.
3/6 figures collected.


 1605 2209.05775
['source_files/2209.05775\\main.tex']
3/3 tables collected.
11/13 figures collected.


 1606 2209.05777
['source_files/2209.05777\\acronyms.tex', 'source_files/2209.05777\\main.tex']
2/2 tables collected.
6/8 figures collected.


 1607 2209.05779
['source_files/2209.05779\\samplepaper.tex']
0/2 tables collected.
0/3 figures collected.


 1608 2209.05794
['source_files/2209.05794\\elsarticle-template-num.tex']
2/3 tables collected.
10/16 figures collected.


 1609 2209.05799
['source_files/2209.05799\\main.tex']
8/13 tables collected.
2/6 figures collected.


 1610 2209.05808
['source_files/2209.05808

8/10 tables collected.
0/16 figures collected.


 1640 2205.04240
['source_files/2205.04240\\main.tex']
0/0 tables collected.
0/0 figures collected.


 1641 2205.08746
['source_files/2205.08746\\ddpce4hs_elsevier-ARXIV.tex', 'source_files/2205.08746\\elsarticle-template.tex', 'source_files/2205.08746\\elsarticle-template\\elsarticle-template.tex', 'source_files/2205.08746\\fig\\DP20_temp.tex', 'source_files/2205.08746\\fig\\DP20_temp_1.tex', 'source_files/2205.08746\\fig\\DP20_Zth.tex', 'source_files/2205.08746\\fig\\IGBT_module.tex', 'source_files/2205.08746\\fig\\IGBT_module_paper.tex', 'source_files/2205.08746\\fig\\transientSobol_paper.tex', 'source_files/2205.08746\\fig\\UQFoster_method4_rel_max.tex']
4/8 tables collected.
0/5 figures collected.


 1642 2206.01589
['source_files/2206.01589\\bin.tex', 'source_files/2206.01589\\root.tex', 'source_files/2206.01589\\sections\\0_abstract.tex', 'source_files/2206.01589\\sections\\1_introduction.tex', 'source_files/2206.01589\\sections\\

4/5 tables collected.
5/22 figures collected.


 1654 2209.06262
['source_files/2209.06262\\output.tex', 'source_files/2209.06262\\uci.tex', 'source_files/2209.06262\\Unifying Causal Inference and RL Arxiv Submission\\output.tex', 'source_files/2209.06262\\Unifying Causal Inference and RL Arxiv Submission\\uci.tex']
Document header could not be identified.
0/0 tables collected.
4/12 figures collected.


 1655 2209.06273
['source_files/2209.06273\\acl2021.tex', 'source_files/2209.06273\\Capstone Paper\\acl2021.tex']
18/20 tables collected.
8/16 figures collected.


 1656 2209.06274
['source_files/2209.06274\\ms.tex', 'source_files/2209.06274\\supplement.tex']
3/4 tables collected.
8/8 figures collected.


 1657 2209.06285
['source_files/2209.06285\\main.tex', 'source_files/2209.06285\\supplementary.tex']
0/2 tables collected.
5/5 figures collected.


 1658 2209.06295
['source_files/2209.06295\\acl.tex']
4/6 tables collected.
1/2 figures collected.


 1659 2209.06302
['source_files/2209.

9/13 tables collected.
2/5 figures collected.


 1681 2209.06470
['source_files/2209.06470\\acl.tex']
4/10 tables collected.
5/12 figures collected.


 1682 2209.06478
['source_files/2209.06478\\main.tex', 'source_files/2209.06478\\src\\00_abstract.tex', 'source_files/2209.06478\\src\\00_acronyms.tex', 'source_files/2209.06478\\src\\00_concepts_and_keywords.tex', 'source_files/2209.06478\\src\\01_introduction.tex', 'source_files/2209.06478\\src\\02_background.tex', 'source_files/2209.06478\\src\\03_methodology.tex', 'source_files/2209.06478\\src\\04_experiments.tex', 'source_files/2209.06478\\src\\05_related_work.tex', 'source_files/2209.06478\\src\\06_conclusions.tex', 'source_files/2209.06478\\src\\07_acknowledgments.tex']
1/1 tables collected.
3/4 figures collected.


 1683 2209.06495
['source_files/2209.06495\\acsa-scn-07.tex', 'source_files/2209.06495\\Self‐organizing life cycle management of mobile ad hoc networks\\acsa-scn-07.tex']
0/0 tables collected.
7/14 figures collected.



1/1 tables collected.
1/6 figures collected.


 1707 2209.06717
['source_files/2209.06717\\main.tex', 'source_files/2209.06717\\regex_task1.tex', 'source_files/2209.06717\\regex_task2.tex', 'source_files/2209.06717\\figures\\dist_bar_plot.tex', 'source_files/2209.06717\\figures\\OOV_len_dist.tex', 'source_files/2209.06717\\figures\\spatial_dist.tex', 'source_files/2209.06717\\plots\\length_analysis_crops.tex', 'source_files/2209.06717\\plots\\length_analysis_e2e.tex', 'source_files/2209.06717\\sota\\sota.tex', 'source_files/2209.06717\\tables\\dataset_stats.tex', 'source_files/2209.06717\\tables\\e2e.tex', 'source_files/2209.06717\\tables\\recognition.tex']
2/3 tables collected.
2/7 figures collected.


 1708 2209.06730
['source_files/2209.06730\\main.tex', 'source_files/2209.06730\\tables\\stvqa-iid.tex', 'source_files/2209.06730\\tables\\stvqa-only_english.tex', 'source_files/2209.06730\\tables\\stvqa-zero_shot.tex', 'source_files/2209.06730\\tables\\stvqa_translate_robustness-tabula